<a href="https://colab.research.google.com/github/mukul-mschauhan/GenerativeAI/blob/main/RAG_on_Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -qU langchain langchain-openai langchain-community chromadb openpyxl
!pip install -qU langchain-classic

In [5]:
import pandas as pd
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_classic.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import os

# SET YOUR API KEY
from google.colab import userdata
api = userdata.get("OPENAI_API_KEY")
#os.environ["OPENAI_API_KEY"] ="OPENAI_API_KEY"

In [6]:
# --- STEP 2: LOAD & FLATTEN EXCEL ---

# We will upload an excel file with columns: [TransactionID, Date, Vendor, Amount, ApprovedBy]
def load_excel_to_documents(file_path):
    df = pd.read_excel(file_path)

    # Handle missing values to avoid errors
    df = df.fillna("N/A")

    documents = []
    for index, row in df.iterrows():
        # CRITICAL STEP: Flatten the row into a single string with context
        # We explicitly repeat the column headers for every value.
        content = f"""
        Transaction Record: {index}
        Transaction ID: {row['TransactionID']}
        Date: {row['Date']}
        Vendor: {row['Vendor']}
        Amount: {row['Amount']}
        Approved By: {row['ApprovedBy']}
        """
        # Store metadata so we can filter later if needed (e.g., filter by Year)
        metadata = {"source": file_path, "row_index": index}

        doc = Document(page_content=content, metadata=metadata)
        documents.append(doc)

    return documents

In [7]:
# Loading the excel file..
docs = load_excel_to_documents("/content/Sample RAG File.xlsx")

In [11]:
docs

[Document(metadata={'source': '/content/Sample RAG File.xlsx', 'row_index': 0}, page_content='\n        Transaction Record: 0\n        Transaction ID: INV-001\n        Date: 2024-01-01 00:00:00\n        Vendor: Acme Corp\n        Amount: 5000\n        Approved By: John Smith\n        '),
 Document(metadata={'source': '/content/Sample RAG File.xlsx', 'row_index': 1}, page_content='\n        Transaction Record: 1\n        Transaction ID: INV-002\n        Date: 2024-01-02 00:00:00\n        Vendor: Global Tech\n        Amount: 12000\n        Approved By: Sarah Lee\n        '),
 Document(metadata={'source': '/content/Sample RAG File.xlsx', 'row_index': 2}, page_content='\n        Transaction Record: 2\n        Transaction ID: INV-003\n        Date: 2024-01-05 00:00:00\n        Vendor: Acme Corp\n        Amount: 2500\n        Approved By: Adam\n        ')]

In [13]:
# --- STEP 3: INDEXING (VECTOR STORE) ---
# We use OpenAI Embeddings to turn the text into numbers
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key=api)

# Create the Vector DB (Chroma)
vector_db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name="audit_records"
)

# --- STEP 4: RETRIEVAL ---
# Create the Retriever interface
retriever = vector_db.as_retriever(search_kwargs={"k": 3}) # Get top 3 matches

In [17]:
# Create the Chat Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0, api_key = api),
    chain_type="stuff",
    retriever=retriever
)

# --- STEP 5: TEST ---
query = "On which date the payment was released to Acme Corp, what was the amount and who approved it?"
response = qa_chain.invoke(query)

print(f"Query: {query}")
print(f"Answer: {response['result']}")

Query: On which date the payment was released to Acme Corp, what was the amount and who approved it?
Answer: The payments to Acme Corp were released on two different dates:

1. On 2024-01-01, the amount was 5000, and it was approved by John Smith.
2. On 2024-01-05, the amount was 2500, and it was approved by Adam.


In [18]:
query = "Give me a summary of the entire excel sheet. What is this sheet all about"
response = qa_chain.invoke(query)

print(f"Query: {query}")
print(f"Answer: {response['result']}")

Query: Give me a summary of the entire excel sheet. What is this sheet all about
Answer: The Excel sheet contains records of financial transactions. Each transaction record includes a unique Transaction ID, the date of the transaction, the vendor involved, the amount of money transacted, and the name of the person who approved the transaction. There are three transactions listed: one with Acme Corp for $5000 approved by John Smith, another with Global Tech for $12000 approved by Sarah Lee, and a second transaction with Acme Corp for $2500 approved by Adam.


In [19]:
query = "Is there any transaction in the name of Oscrop. Whats the amount"
response = qa_chain.invoke(query)

print(f"Query: {query}")
print(f"Answer: {response['result']}")

Query: Is there any transaction in the name of Oscrop. Whats the amount
Answer: I don't have any information on a transaction involving Oscrop. The transactions I have are with Acme Corp and Global Tech.
